# 三 PyTorch的主要组成模块

## 1 完成深度学习的必要部分

### 1.1 机器学习重要步骤
- 数据预处理
  1. 数据格式的统一
  2. 异常数据的消除
  3. 必要的数据变换
  4. 划分训练集、验证集、测试集
- 模型设计
  1. 模型选择
  2. 设定损失函数、优化方法、超参数
- 模型训练、评估

### 1.2 深度学习重要步骤
- 数据预处理：通过专门的数据加载，通过批（batch）训练等策略，每次训练读取固定数量的样本送入模型中训练
- 神经网络搭建：在模型实现上“逐层”搭建，或者预先定义好可以实现特定功能的模块，再把这些模块组装起来
- 损失函数和优化器设定：要能够保证反向传播能够在用户自行定义的模型结构上实现
- 模型训练：每次读入一个批次的数据，放入GPU中训练，然后将损失函数反向传播回网络最前面的层，同时使用优化器调整网络参数。训练/验证后还需要根据设定好的指标计算模型表现，并保存模型参数

## 2 基本配置

- 导入必须的包

In [82]:
import os 
import numpy as np 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optimizer

- 设置超参数

In [83]:
# 批次的大小
batch_size = 16
# 优化器的学习率
lr = 1e-4
# 训练次数
max_epochs = 100
# 配置GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # 指明调用的GPU为1号

In [84]:
# 除了直接将超参数设置在训练的代码里，我们也可以使用yaml、json，dict等文件来存储超参数
# 方案一：使用os.environ，这种情况如果使用GPU不需要设置
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' # 指明调用的GPU为0,1号

# 方案二：使用“device”，后续对要使用GPU的变量用.to(device)即可
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # 指明调用的GPU为1号

## 3 数据读入

数据读入：通过`Dataset`+`DataLoader`的方式完成的，Dataset定义好数据的格式和数据变换形式，DataLoader用iterative的方式不断读入批次数据。

数据变换：TorchVision 数据集都有两个参数: [`transform`](https://pytorch.org/vision/stable/transforms.html) 用于修改特征和 target_transform 用于修改标签


### 3.1 Dataset 定义数据

- 常见的数据读取方式(如[`torchvision.datasets`](https://pytorch.org/vision/stable/datasets.html)、[`torchtext.datasets`](https://pytorch.org/text/stable/datasets.html)、[`torchaudio.datasets`](https://pytorch.org/audio/stable/datasets.html))

```python
import torch
from torchvision import datasets
train_data = datasets.ImageFolder(train_path, transform=data_transform)
val_data = datasets.ImageFolder(val_path, transform=data_transform)

```

- 自己定制Dataset类
  - 实现`__init___`、`__getitem__`、`__len__`函数

```python
import os
import pandas as pd
from torchvision.io import read_image

class MyDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        """
        Args:
            annotations_file (string): Path to the csv file with annotations.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            target_transform (callable, optional): Optional transform to be applied
                on the target.
        """
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index
        """
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

```

### 3.2 DataLoader 读入数据

```python
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=4, shuffle=False)

```

- batch_size：样本是按“批”读入的，batch_size就是每次读入的样本数
- num_workers：有多少个进程用于读取数据，Windows下该参数设置为0，Linux下常见的为4或者8，根据自己的电脑配置来设置
- shuffle：是否将读入的数据打乱，一般在训练集中设置为True，验证集中设置为False
- drop_last：对于样本最后一部分没有达到批次数的样本，使其不再参与训练

这里可以看一下我们的加载的数据。PyTorch中的DataLoader的读取可以使用next和iter来完成

```python
import matplotlib.pyplot as plt
images, labels = next(iter(val_loader))
print(images.shape)
plt.imshow(images[0].transpose(1,2,0))
plt.show()

```

### 3.3 transform 数据变换

```python
# 设置数据变换
from torchvision import transforms

image_size = 28
data_transform = transforms.Compose([
    transforms.ToPILImage(),  
    transforms.Resize(image_size),
    transforms.ToTensor()
])
```

## 4 模型构建

### 4.1 神经网络的构造

`Module` 类是 `torch.nn` 模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型

下面定义的 MLP 类重载了 `Module` 类的 `__init__` 函数和 `forward` 函数。⽆须定义反向传播函数。系统将通过⾃动求梯度⽽自动⽣成反向传播所需的 `backward` 函数。

In [7]:
import torch
from torch import nn

class MLP(nn.Module):
  # 声明带有模型参数的层，这里声明了两个全连接层
  def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Linear(784, 256)
    self.act = nn.ReLU()
    self.output = nn.Linear(256,10)
    
    # self.linear_relu_stack = nn.Sequential(
    #         nn.Linear(28*28, 512),
    #         nn.ReLU(),
    #         nn.Linear(512, 512),
    #         nn.ReLU(),
    #         nn.Linear(512, 10)
    #     )
    
   # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
  def forward(self, x):
    o = self.act(self.hidden(x))
    return self.output(o)   

In [9]:
# 获取 cpu, gpu 或 mps 设备用于加速训练.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

X = torch.rand(2,784).to(device) # 设置一个随机的输入张量
net = MLP().to(device) # 实例化模型
print(net) # 打印模型
net(X) # 前向计算，调用MLP的forward函数


Using mps device
MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.2240, -0.0688, -0.0741, -0.0647, -0.0319,  0.3036,  0.0677,  0.1490,
         -0.0089,  0.0076],
        [-0.2055, -0.0656, -0.0038, -0.2204,  0.0974,  0.2616, -0.0102,  0.2200,
         -0.0183, -0.1034]], device='mps:0', grad_fn=<LinearBackward0>)

### 4.2 神经网络中常见的层

- 神经网络中各式各样的层，例如全连接层、卷积层、池化层与循环层等。这些层在 PyTorch 中都有相应的类可以实现，例如`nn.Flatten` `nn.Linear`、`nn.Conv2d`、`nn.ReLU`、`nn.Sequential`、`nn.Softmax`、`nn.MaxPool2d` 和 `nn.LSTM` 等。

除了神经网络层还有其它分层结构如：
- 激活函数层：ReLU、Sigmoid、Tanh、LeakyReLU、Softmax等。
- 损失函数层：均方误差、交叉熵、Hinge Loss、多标签损失等。
- 优化器层：SGD、Adam、RMSProp等。
- 预处理层：Normalize、ToTensor、RandomCrop、RandomHorizontalFlip等。
- 模型层：AlexNet、VGG、ResNet、Inception、DenseNet等。
- 数据层：MNIST、CIFAR10、CIFAR100、ImageNet等。
- 数据集层：CIFAR10Data、MNISTData、CIFAR100Data、ImageNetData等。
- 数据加载器层：DataLoader、DataLoader2等。
- 模型加载器层：ModelLoader等。
- 模型保存器层：ModelSaver等。

#### 4.2.1 不含模型参数的层

In [87]:
import torch
from torch import nn

class MyLayer(nn.Module):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()  

layer = MyLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))


tensor([-2., -1.,  0.,  1.,  2.])

#### 4.2.2 含模型参数的层
我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

`Parameter` 类其实是 `Tensor` 的子类，如果一个 `Tensor` 是 `Parameter` ，那么它会⾃动被添加到模型的参数列表里。所以在⾃定义含模型参数的层时，我们应该将参数定义成 `Parameter` ，除了直接定义成 `Parameter` 类外，还可以使⽤ `ParameterList` 和 `ParameterDict` 分别定义参数的列表和字典。

In [88]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4]
      (1): Parameter containing: [torch.float32 of size 4x4]
      (2): Parameter containing: [torch.float32 of size 4x4]
      (3): Parameter containing: [torch.float32 of size 4x1]
  )
)


In [89]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


#### 4.2.3 二维卷积层

下面给出常见的神经网络的一些层，比如卷积层、池化层，以及较为基础的AlexNet，LeNet等。

二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

In [90]:
import torch
from torch import nn

# 卷积运算（二维互相关）
def corr2d(X, K): 
    h, w = K.shape
    X, K = X.float(), K.float()
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

# 定义二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [91]:
import torch
from torch import nn

# 定义一个函数来计算卷积层。它对输入和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
    print("before:", X.shape)
    # (1, 1)代表批量大小和通道数
    X = X.view((1, 1) + X.shape)
    print("after:", X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:]) # 排除不关心的前两维:批量和通道

# 注意这里是两侧分别填充1⾏或列，所以在两侧一共填充2⾏或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3,padding=1)

X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

before: torch.Size([8, 8])
after: torch.Size([1, 1, 8, 8])


torch.Size([8, 8])

当卷积核的高和宽不同时，我们也可以通过设置高和宽上不同的填充数使输出和输入具有相同的高和宽。

In [92]:
# 使用高为5、宽为3的卷积核。在⾼和宽两侧的填充数分别为2和1
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

before: torch.Size([8, 8])
after: torch.Size([1, 1, 8, 8])


torch.Size([8, 8])

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下 的顺序，依次在输⼊数组上滑动。我们将每次滑动的行数和列数称为步幅(stride)。

In [93]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

before: torch.Size([8, 8])
after: torch.Size([1, 1, 8, 8])


torch.Size([2, 2])

填充可以增加输出的高和宽。这常用来使输出与输入具有相同的高和宽。

步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的 ( 为大于1的整数)。

#### 4.2.4 池化层

池化层每次对输入数据的一个固定形状窗口(⼜称池化窗口)中的元素计算输出（均值、最大值等。常见的池化包括最大池化或平均池化。

下面把池化层的前向计算实现在 `pool2d`函数里。

In [94]:
import torch
from torch import nn

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [95]:
# 默认的最大池化
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]], dtype=torch.float)
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [96]:
#平均池化
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### 4.3 模型示例

我们可以使用 `torch.nn`包来构建神经网络。我们已经介绍了 `autograd`包，`nn`包则依赖于 `autograd`包来定义模型并对它们求导。一个 `nn.Module`包含各个层和一个 `forward(input)`方法，该方法返回 `output`。

- 一个神经网络的典型训练过程如下：

1. 定义包含一些可学习参数(或者叫权重）的神经网络
2. 在输入数据集上迭代
3. 通过网络处理输入
4. 计算 loss (输出和正确答案的距离）
5. 将梯度反向传播给网络的参数
6. 更新网络的权重，一般使用一个简单的规则：`weight = weight - learning_rate * gradient`

#### 4.3.1 LeNet

这是一个简单的前馈神经网络 (feed-forward network）（LeNet）。它接受一个输入，然后将它送入下一层，一层接一层的传递，最后给出输出。

In [97]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入图像channel是1；输出channel是6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果是方阵,则可以只使用一个数字进行定义
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 除去批处理维度的其他所有维度
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [98]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [99]:
# 一个模型的可学习参数可以通过 `net.parameters()`返回
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1的权重

10
torch.Size([6, 1, 5, 5])


In [100]:
# 假设输入的数据为随机的32x32
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0347, -0.0997,  0.1539, -0.0471, -0.0579, -0.0295,  0.0146,  0.0114,
         -0.0535, -0.0199]], grad_fn=<AddmmBackward0>)


In [101]:
# 清零所有参数的梯度缓存，然后进行随机梯度的反向传播
net.zero_grad()
out.backward(torch.randn(1, 10))
print(net.conv1.bias.grad)

tensor([ 0.0955, -0.0494, -0.0218, -0.0334,  0.0064,  0.0633])


- `torch.Tensor` - 一个多维数组，支持诸如 `backward()`等的自动求导操作，同时也保存了张量的梯度。
- `nn.Module `- 神经网络模块。是一种方便封装参数的方式，具有将参数移动到GPU、导出、加载等功能。
- `nn.Parameter `- 张量的一种，当它作为一个属性分配给一个 `Module`时，它会被自动注册为一个参数。
- `autograd.Function` - 实现了自动求导前向和反向传播的定义，每个 `Tensor`至少创建一个 `Function`节点，该节点连接到创建 `Tensor`的函数并对其历史进行编码。

#### 4.3.2 AlexNet

In [102]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
         # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [103]:
net = AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

## 5 模型初始化

PyTorch也在 `torch.nn.init`中为我们提供了常用的初始化方法。

### 5.1 torch.nn.init内容

通过访问torch.nn.init的官方文档[链接](https://pytorch.org/docs/stable/nn.init.html) ，我们发现`torch.nn.init`提供了以下初始化方法：
1. `torch.nn.init.uniform_`(tensor, a=0.0, b=1.0)
2. `torch.nn.init.normal_`(tensor, mean=0.0, std=1.0)
3. `torch.nn.init.constant_`(tensor, val)
4. `torch.nn.init.ones_`(tensor)
5. `torch.nn.init.zeros_`(tensor)
6. `torch.nn.init.eye_`(tensor)
7. `torch.nn.init.dirac_`(tensor, groups=1)
8. `torch.nn.init.xavier_uniform_`(tensor, gain=1.0)
9. `torch.nn.init.xavier_normal_`(tensor, gain=1.0)
10. `torch.nn.init.kaiming_uniform_`(tensor, a=0, mode='fan__in', nonlinearity='leaky_relu')
11. `torch.nn.init.kaiming_normal_`(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
12. `torch.nn.init.orthogonal_`(tensor, gain=1)
13. `torch.nn.init.sparse_`(tensor, sparsity, std=0.01)
14.  `torch.nn.init.calculate_gain`(nonlinearity, param=None)

### 5.2 torch.nn.init使用

我们通常会根据实际模型来使用 `torch.nn.init`进行初始化，通常使用 `isinstance()`来进行判断模块（回顾3.4模型构建）属于什么类型。

In [104]:
import torch
import torch.nn as nn

conv = nn.Conv2d(1,3,3)
linear = nn.Linear(10,1)

print(isinstance(conv,nn.Conv2d)) # 判断conv是否是nn.Conv2d类型
print(isinstance(linear,nn.Conv2d)) # 判断linear是否是nn.Conv2d类型

True
False


In [105]:
# 查看随机初始化的conv参数
print(conv.weight.data)
# 查看linear的参数
print(linear.weight.data)

tensor([[[[ 0.1214,  0.2806, -0.0292],
          [ 0.1002, -0.0138, -0.1674],
          [-0.2870, -0.0321,  0.0098]]],


        [[[-0.2423, -0.1535, -0.1605],
          [ 0.2817, -0.0508, -0.2556],
          [-0.1427,  0.2468,  0.2649]]],


        [[[-0.3123,  0.1491, -0.0726],
          [ 0.1039, -0.3151,  0.1846],
          [-0.1921, -0.1371, -0.1748]]]])
tensor([[ 0.0097, -0.0638, -0.0207,  0.0542, -0.0961, -0.1023, -0.1937,  0.3078,
          0.3067,  0.1918]])


In [106]:
# 现在对conv进行kaiming初始化
torch.nn.init.kaiming_normal_(conv.weight.data)
print(conv.weight.data)
# 对linear进行常数初始化
torch.nn.init.constant_(linear.weight.data,0.3)
print(linear.weight.data)

tensor([[[[-0.0098,  0.1265, -0.6941],
          [ 0.6194, -0.0978,  0.9692],
          [-0.0849,  0.5016,  0.4319]]],


        [[[ 0.3509, -0.4570,  1.2782],
          [ 0.1659, -0.3807, -0.9668],
          [ 0.0276, -0.8417,  0.9629]]],


        [[[-0.4879, -0.2761, -0.4743],
          [ 0.0036,  1.3106, -0.1651],
          [ 0.0342, -0.1146, -0.3410]]]])
tensor([[0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
         0.3000]])


### 5.3 对初始化函数的调用进行封装

人们常常将各种初始化方法定义为一个 `initialize_weights()`的函数并在模型初始后进行使用。

In [107]:
# 遍历当前模型的每一层，判断不同类型，设定不同的权值初始化方法。
def initialize_weights(model):
	for m in model.modules():
		# 判断是否属于Conv2d
		if isinstance(m, nn.Conv2d):
			torch.nn.init.zeros_(m.weight.data)
			# 判断是否有偏置
			if m.bias is not None:
				torch.nn.init.constant_(m.bias.data,0.3)
		elif isinstance(m, nn.Linear):
			torch.nn.init.normal_(m.weight.data, 0.1)
			if m.bias is not None:
				torch.nn.init.zeros_(m.bias.data)
		elif isinstance(m, nn.BatchNorm2d):
			m.weight.data.fill_(1) 		 
			m.bias.data.zeros_()	


In [108]:
# 演示
# 模型的定义
class MLP(nn.Module):
  # 声明带有模型参数的层，这里声明了两个全连接层
  def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Conv2d(1,1,3)
    self.act = nn.ReLU()
    self.output = nn.Linear(10,1)
    
   # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
  def forward(self, x):
    o = self.act(self.hidden(x))
    return self.output(o)

mlp = MLP()
print(mlp.hidden.weight.data)
print("-------初始化-------")

mlp.apply(initialize_weights)
# 或者initialize_weights(mlp)
print(mlp.hidden.weight.data)


tensor([[[[-0.1459, -0.2845, -0.1582],
          [ 0.0229, -0.1054, -0.1119],
          [ 0.1270, -0.2003, -0.1569]]]])
-------初始化-------
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])


**注意：**
我们在初始化时，最好不要将模型的参数初始化为0，因为这样会导致梯度消失，从而影响模型的训练效果。因此，我们在初始化时，可以使用其他初始化方法或者将模型初始化为一个很小的值，如0.01，0.1等。

## 6 损失函数

### 6.1 二分类交叉熵损失函数 nn.BCELoss

```python
torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
```

**功能：** 计算二分类任务时的交叉熵（Cross Entropy）函数。在二分类中，label是{0,1}。对于进入交叉熵函数的input为概率分布的形式。一般来说，input为sigmoid激活层的输出，或者softmax的输出。

**主要参数**：

`weight`:每个类别的loss设置权值

`size_average`:数据为bool，为True时，返回的loss为平均值；为False时，返回的各样本的loss之和。

`reduce`:数据类型为bool，为True时，loss的返回是标量。

```python
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
output.backward()
print('BCELoss损失函数的计算结果为',output)
```

BCELoss损失函数的计算结果为 tensor(0.5732, grad_fn=`<BinaryCrossEntropyBackward>`)

### 6.2 交叉熵损失函数 nn.CrossEntropyLoss

```python
torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
```

**功能**：计算交叉熵函数

**主要参数**：

`weight`:每个类别的loss设置权值。

`size_average`:数据为bool，为True时，返回的loss为平均值；为False时，返回的各样本的loss之和。

`ignore_index`:忽略某个类的损失函数。

`reduce`:数据类型为bool，为True时，loss的返回是标量。

```python
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
```

tensor(2.0115, grad_fn=`<NllLossBackward>`)

### 6.3 L1损失函数 nn.L1Loss

```python
torch.nn.L1Loss(size_average=None, reduce=None, reduction='mean')
```

**功能：** 计算输出 `y`和真实标签 `target`之间的差值的绝对值。

`reduction`参数决定了计算模式。有三种计算模式可选：none：逐个元素计算。
sum：所有元素求和，返回标量。
mean：加权平均，返回标量。
如果选择 `none`，那么返回的结果是和输入元素相同尺寸的。默认计算方式是求平均。

```python
loss = nn.L1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()
print('L1损失函数的计算结果为',output)
```

L1损失函数的计算结果为 tensor(1.5729, grad_fn=`<L1LossBackward>`)

### 6.4 MSE损失函数 nn.MSELoss

```python
torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')
```

**功能：** 计算输出 `y`和真实标签 `target`之差的平方。

```python
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()
print('MSE损失函数的计算结果为',output)
```

MSE损失函数的计算结果为 tensor(1.6968, grad_fn=`<MseLossBackward>`)

### 6.5 平滑L1 (Smooth L1)损失函数 nn.SmoothL1Loss

```python
torch.nn.SmoothL1Loss(size_average=None, reduce=None, reduction='mean', beta=1.0)
```

**功能：** L1的平滑输出，其功能是减轻离群点带来的影响

```python
loss = nn.SmoothL1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()
print('SmoothL1Loss损失函数的计算结果为',output)
```

SmoothL1Loss损失函数的计算结果为 tensor(0.7808, grad_fn=`<SmoothL1LossBackward>`)

### 6.6 目标泊松分布的负对数似然损失 nn.PoissonNLLLoss

```python
torch.nn.PoissonNLLLoss(log_input=True, full=False, size_average=None, eps=1e-08, reduce=None, reduction='mean')
```

**功能：** 泊松分布的负对数似然损失函数

**主要参数：**

`log_input`：输入是否为对数形式，决定计算公式。

`full`：计算所有 loss，默认为 False。

`eps`：修正项，避免 input 为 0 时，log(input) 为 nan 的情况。

```python
loss = nn.PoissonNLLLoss()
log_input = torch.randn(5, 2, requires_grad=True)
target = torch.randn(5, 2)
output = loss(log_input, target)
output.backward()
print('PoissonNLLLoss损失函数的计算结果为',output)
```

```
PoissonNLLLoss损失函数的计算结果为 tensor(0.7358, grad_fn=<MeanBackward0>)
```

### 6.7 KL散度 nn.KLDivLoss

```python
torch.nn.KLDivLoss(size_average=None, reduce=None, reduction='mean', log_target=False)
```

**功能：** 计算KL散度，也就是计算相对熵。用于连续分布的距离度量，并且对离散采用的连续输出空间分布进行回归通常很有用。

```python
inputs = torch.tensor([[0.5, 0.3, 0.2], [0.2, 0.3, 0.5]])
target = torch.tensor([[0.9, 0.05, 0.05], [0.1, 0.7, 0.2]], dtype=torch.float)
loss = nn.KLDivLoss()
output = loss(inputs,target)

print('KLDivLoss损失函数的计算结果为',output)

```

KLDivLoss损失函数的计算结果为 tensor(-0.3335)

### 6.8 MarginRankingLoss

```python
torch.nn.MarginRankingLoss(margin=0.0, size_average=None, reduce=None, reduction='mean')
```

**功能：** 计算两个向量之间的相似度，用于排序任务。该方法用于计算两组数据之间的差异。

**主要参数:**

`margin`：边界值，$x_{1}$ 与$x_{2}$ 之间的差异值。

`reduction`：计算模式，可为 none/sum/mean。

```python
loss = nn.MarginRankingLoss()
input1 = torch.randn(3, requires_grad=True)
input2 = torch.randn(3, requires_grad=True)
target = torch.randn(3).sign()
output = loss(input1, input2, target)
output.backward()

print('MarginRankingLoss损失函数的计算结果为',output)

```

MarginRankingLoss损失函数的计算结果为 tensor(0.7740, grad_fn=`<MeanBackward0>`)

### 6.9 多标签边界损失函数 nn.MultiLabelMarginLoss

```python
torch.nn.MultiLabelMarginLoss(size_average=None, reduce=None, reduction='mean')
```

**功能：** 对于多标签分类问题计算损失函数。

```python
loss = nn.MultiLabelMarginLoss()
x = torch.FloatTensor([[0.9, 0.2, 0.4, 0.8]])
# for target y, only consider labels 3 and 0, not after label -1
y = torch.LongTensor([[3, 0, -1, 1]])# 真实的分类是，第3类和第0类
output = loss(x, y)

print('MultiLabelMarginLoss损失函数的计算结果为',output)
```

MultiLabelMarginLoss损失函数的计算结果为 tensor(0.4500)

### 6.10 二分类损失函数 nn.SoftMarginLoss

```python
torch.nn.SoftMarginLoss(size_average=None, reduce=None, reduction='mean')torch.nn.(size_average=None, reduce=None, reduction='mean')
```

**功能：** 计算二分类的 logistic 损失。

```python
inputs = torch.tensor([[0.3, 0.7], [0.5, 0.5]])  # 两个样本，两个神经元
target = torch.tensor([[-1, 1], [1, -1]], dtype=torch.float)  # 该 loss 为逐个神经元计算，需要为每个神经元单独设置标签

loss_f = nn.SoftMarginLoss()
output = loss_f(inputs, target)

print('SoftMarginLoss损失函数的计算结果为',output)

```

SoftMarginLoss损失函数的计算结果为 tensor(0.6764)

### 6.11 多分类的折页损失 nn.MultiMarginLoss

```python
torch.nn.MultiMarginLoss(p=1, margin=1.0, weight=None, size_average=None, reduce=None, reduction='mean')
```

**功能：** 计算多分类的折页损失

**主要参数:**

`p：`可选 1 或 2。

```python
inputs = torch.tensor([[0.3, 0.7], [0.5, 0.5]]) 
target = torch.tensor([0, 1], dtype=torch.long) 

loss_f = nn.MultiMarginLoss()
output = loss_f(inputs, target)

print('MultiMarginLoss损失函数的计算结果为',output)

```

MultiMarginLoss损失函数的计算结果为 tensor(0.6000)

### 6.12 三元组损失 nn.TripletMarginLoss

```python
torch.nn.TripletMarginLoss(margin=1.0, p=2.0, eps=1e-06, swap=False, size_average=None, reduce=None, reduction='mean')

```

**功能：** 计算三元组损失。

**三元组:** 这是一种数据的存储或者使用格式。<实体1，关系，实体2>。在项目中，也可以表示为< `anchor`, `positive examples` , `negative examples`>

在这个损失函数中，我们希望去 `anchor`的距离更接近 `positive examples`，而远离 `negative examples `

```python
triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
anchor = torch.randn(100, 128, requires_grad=True)
positive = torch.randn(100, 128, requires_grad=True)
negative = torch.randn(100, 128, requires_grad=True)
output = triplet_loss(anchor, positive, negative)
output.backward()
print('TripletMarginLoss损失函数的计算结果为',output)

```

TripletMarginLoss损失函数的计算结果为 tensor(1.1667, grad_fn=`<MeanBackward0>`)

### 6.13 HingEmbeddingLoss nn.HingeEmbeddingLoss

```python
torch.nn.HingeEmbeddingLoss(margin=1.0, size_average=None, reduce=None, reduction='mean')
```

**功能：** 对输出的embedding结果做Hing损失计算

```python
loss_f = nn.HingeEmbeddingLoss()
inputs = torch.tensor([[1., 0.8, 0.5]])
target = torch.tensor([[1, 1, -1]])
output = loss_f(inputs,target)

print('HingEmbeddingLoss损失函数的计算结果为',output)

```

HingEmbeddingLoss损失函数的计算结果为 tensor(0.7667)

### 6.14 余弦相似度 nn.CosineEmbeddingLoss

```python
torch.nn.CosineEmbeddingLoss(margin=0.0, size_average=None, reduce=None, reduction='mean')

```

**功能：** 对两个向量做余弦相似度

```python
loss_f = nn.CosineEmbeddingLoss()
inputs_1 = torch.tensor([[0.3, 0.5, 0.7], [0.3, 0.5, 0.7]])
inputs_2 = torch.tensor([[0.1, 0.3, 0.5], [0.1, 0.3, 0.5]])
target = torch.tensor([1, -1], dtype=torch.float)
output = loss_f(inputs_1,inputs_2,target)

print('CosineEmbeddingLoss损失函数的计算结果为',output)

```

CosineEmbeddingLoss损失函数的计算结果为 tensor(0.5000)

### 6.15 CTC损失函数 nn.CTCLoss

```python
torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=False)

```

**功能：** 用于解决时序类数据的分类

**主要参数:**

`reduction`：计算模式，可为 none/sum/mean。

`blank`：blank label。

`zero_infinity`：无穷大的值或梯度值为

```python
# Target are to be padded
T = 50      # Input sequence length
C = 20      # Number of classes (including blank)
N = 16      # Batch size
S = 30      # Target sequence length of longest target in batch (padding length)
S_min = 10  # Minimum target length, for demonstration purposes

# Initialize random batch of input vectors, for *size = (T,N,C)
input = torch.randn(T, N, C).log_softmax(2).detach().requires_grad_()

# Initialize random batch of targets (0 = blank, 1:C = classes)
target = torch.randint(low=1, high=C, size=(N, S), dtype=torch.long)

input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.long)
target_lengths = torch.randint(low=S_min, high=S, size=(N,), dtype=torch.long)
ctc_loss = nn.CTCLoss()
loss = ctc_loss(input, target, input_lengths, target_lengths)
loss.backward()


# Target are to be un-padded
T = 50      # Input sequence length
C = 20      # Number of classes (including blank)
N = 16      # Batch size

# Initialize random batch of input vectors, for *size = (T,N,C)
input = torch.randn(T, N, C).log_softmax(2).detach().requires_grad_()
input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.long)

# Initialize random batch of targets (0 = blank, 1:C = classes)
target_lengths = torch.randint(low=1, high=T, size=(N,), dtype=torch.long)
target = torch.randint(low=1, high=C, size=(sum(target_lengths),), dtype=torch.long)
ctc_loss = nn.CTCLoss()
loss = ctc_loss(input, target, input_lengths, target_lengths)
loss.backward()

print('CTCLoss损失函数的计算结果为',loss)

```

CTCLoss损失函数的计算结果为 tensor(16.0885, grad_fn=`<MeanBackward0>`)

## 7 训练和评估

在完成了模型的训练后，需要在测试集/验证集上完成模型的验证，以确保我们的模型具有泛化能力、不会出现过拟合等问题。在PyTorch中，训练和评估的流程是一致的，只是在训练过程中需要将模型的参数进行更新，而在评估过程中则不需要更新参数。

验证/测试的流程基本与训练过程一致，不同点在于：

- 需要预先设置torch.no_grad，以及将model调至eval模式
- 不需要将优化器的梯度置零
- 不需要将loss反向回传到网络
- 不需要更新optimizer

### 7.1 模型训练

```python
# 定义训练函数
def train(epoch):
    # 设置模型为训练模式
    model.train()
    # 初始化训练损失
    train_loss = 0
    # 遍历训练数据集
    for data, label in train_loader:
        # 将数据和标签转换为cuda格式
        data, label = data.cuda(), label.cuda()
        # 梯度归零
        optimizer.zero_grad()
        # 将data送入模型中训练，计算模型输出
        output = model(data)
        # 计算损失
        loss = criterion(output, label)
        # 反向传播
        loss.backward()
        # 使用优化器更新参数
        optimizer.step()
        # 累加损失
        train_loss += loss.item()*data.size(0)
    # 计算平均损失
    train_loss = train_loss/len(train_loader.dataset)
    # 打印训练损失
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

```

### 7.2 模型验证

```python
# 定义val函数，用于验证模型
def val(epoch):       
    # 将模型设置为评估模式
    model.eval()
    # 初始化验证损失
    val_loss = 0
    # 不计算梯度
    with torch.no_grad():
        # 遍历验证数据集
        for data, label in val_loader:
            # 将数据和标签转换为cuda格式
            data, label = data.cuda(), label.cuda()
            # 运行模型，得到输出
            output = model(data)
            # 得到预测结果
            preds = torch.argmax(output, 1)
            # 计算损失
            loss = criterion(output, label)
            # 累加损失
            val_loss += loss.item()*data.size(0)
            # 累加准确率
            running_accu += torch.sum(preds == label.data)
    # 计算平均损失
    val_loss = val_loss/len(val_loader.dataset)
    # 打印验证损失
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, val_loss))
```

对于图像分类任务，我们还可以使用sklearn.metrics中的classification_report函数来计算模型的准确率、召回率、F1值等指标，如下所示：

```python
from sklearn.metrics import classification_report
"""
将下方代码的labels和preds替换为模型预测出来的所有label和preds，
target_names替换为类别名称，
既可得到模型的分类报告
"""
print(classification_report(labels.cpu(), preds.cpu(), target_names=class_names))

```

除此之外，我们还可以使用 `torcheval`或 `torchmetric`来对模型进行评估。

## 8 可视化

可以对分类的ROC曲线，卷积网络中的卷积核，以及训练/验证过程的损失函数曲线等内容进行可视化。参考第七章。

## 9 PyTorch优化器

优化器是根据网络反向传播的梯度信息来更新网络的参数，以起到降低loss函数计算值，使得模型输出更加接近真实标签。

### 9.1 PyTorch提供的优化器

+ torch.optim.SGD
+ torch.optim.ASGD
+ torch.optim.Adadelta
+ torch.optim.Adagrad
+ torch.optim.Adam
+ torch.optim.AdamW
+ torch.optim.Adamax
+ torch.optim.RAdam
+ torch.optim.NAdam
+ torch.optim.SparseAdam
+ torch.optim.LBFGS
+ torch.optim.RMSprop
+ torch.optim.Rprop

PyTorch提供了一个优化器的库 `torch.optim`包含多种优化器。而以上这些优化算法均继承于 `Optimizer`

```python
class Optimizer(object):
    def __init__(self, params, defaults):        
        self.defaults = defaults
        self.state = defaultdict(dict)
        self.param_groups = []

```

**`Optimizer`有三个属性：**

+ `defaults`：存储的是优化器的超参数，例子如下：

```python
{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}
```

+ `state`：参数的缓存，例子如下：

```python
defaultdict(<class 'dict'>, {tensor([[ 0.3864, -0.0131],
        [-0.1911, -0.4511]], requires_grad=True): {'momentum_buffer': tensor([[0.0052, 0.0052],
        [0.0052, 0.0052]])}})
```

+ `param_groups`：管理的参数组，是一个list，其中每个元素是一个字典，顺序是params，lr，momentum，dampening，weight_decay，nesterov，例子如下：

```python
[{'params': [tensor([[-0.1022, -1.6890],[-1.5116, -1.7846]], requires_grad=True)], 'lr': 1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}]
```

`Optimizer`还有以下的方法：

+ `zero_grad()`：清空所管理参数的梯度，PyTorch的特性是张量的梯度不自动清零，因此每次反向传播后都需要清空梯度。
+ `step()`：执行一步梯度更新，参数更新
+ `add_param_group()`：添加参数组
+ `load_state_dict()` ：加载状态参数字典，可以用来进行模型的断点续训练，继续上次的参数进行训练
+ `state_dict()`：获取优化器当前状态信息字典


### 9.2 实际操作

```python
import os
import torch

# 设置权重，服从正态分布  --> 2 x 2
weight = torch.randn((2, 2), requires_grad=True)
# 设置梯度为全1矩阵  --> 2 x 2
weight.grad = torch.ones((2, 2))
# 输出现有的weight和data
print("The data of weight before step:\n{}".format(weight.data))
print("The grad of weight before step:\n{}".format(weight.grad))
# 实例化优化器
optimizer = torch.optim.SGD([weight], lr=0.1, momentum=0.9)
# 进行一步操作
optimizer.step()
# 查看进行一步后的值，梯度
print("The data of weight after step:\n{}".format(weight.data))
print("The grad of weight after step:\n{}".format(weight.grad))
# 梯度清零
optimizer.zero_grad()
# 检验梯度是否为0
print("The grad of weight after optimizer.zero_grad():\n{}".format(weight.grad))
# 输出参数信息
print("optimizer.params_group is \n{}".format(optimizer.param_groups))
# 查看参数位置，optimizer和weight的位置一样，我觉得这里可以参考Python是基于值管理
print("weight in optimizer:{}\nweight in weight:{}\n".format(id(optimizer.param_groups[0]['params'][0]), id(weight)))
# 添加参数：weight2
weight2 = torch.randn((3, 3), requires_grad=True)
optimizer.add_param_group({"params": weight2, 'lr': 0.0001, 'nesterov': True})
# 查看现有的参数信息
print("optimizer.param_groups is\n{}".format(optimizer.param_groups))
# 查看当前状态信息
opt_state_dict = optimizer.state_dict()
print("state_dict before step:\n", opt_state_dict)
# 进行5次step操作
for _ in range(50):
    optimizer.step()
# 输出现有状态信息
print("state_dict after step:\n", optimizer.state_dict())
# 保存参数信息
torch.save(optimizer.state_dict(),os.path.join(r"D:\pythonProject\Attention_Unet", "optimizer_state_dict.pkl"))
print("----------done-----------")
# 加载参数信息
state_dict = torch.load(r"D:\pythonProject\Attention_Unet\optimizer_state_dict.pkl") # 需要修改为你自己的路径
optimizer.load_state_dict(state_dict)
print("load state_dict successfully\n{}".format(state_dict))
# 输出最后属性信息
print("\n{}".format(optimizer.defaults))
print("\n{}".format(optimizer.state))
print("\n{}".format(optimizer.param_groups))

```

### 9.3 输出结果

```python
# 进行更新前的数据，梯度
The data of weight before step:
tensor([[-0.3077, -0.1808],
        [-0.7462, -1.5556]])
The grad of weight before step:
tensor([[1., 1.],
        [1., 1.]])
# 进行更新后的数据，梯度
The data of weight after step:
tensor([[-0.4077, -0.2808],
        [-0.8462, -1.6556]])
The grad of weight after step:
tensor([[1., 1.],
        [1., 1.]])
# 进行梯度清零的梯度
The grad of weight after optimizer.zero_grad():
tensor([[0., 0.],
        [0., 0.]])
# 输出参数信息
optimizer.params_group is 
[{'params': [tensor([[-0.4077, -0.2808],
        [-0.8462, -1.6556]], requires_grad=True)], 'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}]

# 证明了优化器的和weight的储存是在一个地方，Python基于值管理
weight in optimizer:1841923407424
weight in weight:1841923407424
    
# 添加参数后输出参数信息
optimizer.param_groups is
[{'params': [tensor([[-0.4077, -0.2808],
        [-0.8462, -1.6556]], requires_grad=True)], 'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}, {'params': [tensor([[ 0.4539, -2.1901, -0.6662],
        [ 0.6630, -1.5178, -0.8708],
        [-2.0222,  1.4573,  0.8657]], requires_grad=True)], 'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0}]

# 进行更新前的参数查看，用state_dict
state_dict before step:
 {'state': {0: {'momentum_buffer': tensor([[1., 1.],
        [1., 1.]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'params': [1]}]}
# 进行更新后的参数查看，用state_dict
state_dict after step:
 {'state': {0: {'momentum_buffer': tensor([[0.0052, 0.0052],
        [0.0052, 0.0052]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'params': [1]}]}

# 存储信息完毕
----------done-----------
# 加载参数信息成功
load state_dict successfully
# 加载参数信息
{'state': {0: {'momentum_buffer': tensor([[0.0052, 0.0052],
        [0.0052, 0.0052]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'params': [1]}]}

# defaults的属性输出
{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}

# state属性输出
defaultdict(<class 'dict'>, {tensor([[-1.3031, -1.1761],
        [-1.7415, -2.5510]], requires_grad=True): {'momentum_buffer': tensor([[0.0052, 0.0052],
        [0.0052, 0.0052]])}})

# param_groups属性输出
[{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [tensor([[-1.3031, -1.1761],
        [-1.7415, -2.5510]], requires_grad=True)]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'params': [tensor([[ 0.4539, -2.1901, -0.6662],
        [ 0.6630, -1.5178, -0.8708],
        [-2.0222,  1.4573,  0.8657]], requires_grad=True)]}]
```

**注意：**

1. 每个优化器都是一个类，我们一定要进行实例化才能使用，比如下方实现：

```python
class Net(nn.Moddule):
    ···
net = Net()
optim = torch.optim.SGD(net.parameters(),lr=lr)
optim.step()
```

2. optimizer在一个神经网络的epoch中需要实现下面两个步骤：
   1. 梯度置零
   2. 梯度更新

```python
optimizer = torch.optim.SGD(net.parameters(), lr=1e-5)
for epoch in range(EPOCH):
	...
	optimizer.zero_grad()  #梯度置零
	loss = ...             #计算loss
	loss.backward()        #BP反向传播
	optimizer.step()       #梯度更新
```

3. 给网络不同的层赋予不同的优化器参数。

```python
from torch import optim
from torchvision.models import resnet18

net = resnet18()

optimizer = optim.SGD([
    {'params':net.fc.parameters()},#fc的lr使用默认的1e-5
    {'params':net.layer4[0].conv1.parameters(),'lr':1e-2}],lr=1e-5)

# 可以使用param_groups查看属性
```